In [1]:
%load_ext lab_black
%cd ..

/home/shim/shim/dev/log-analytics


In [2]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    DistilBertForSequenceClassification,
    DistilBertTokenizer,
    RobertaForSequenceClassification,
    RobertaTokenizer,
)
from sklearn.model_selection import StratifiedKFold
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

from datasets import load_train_data
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything
from main import MyTrainer

In [3]:
with open("config/distilbert-base-uncased.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [4]:
C

{'model': {'name': 'distilbert-base-uncased'},
 'comment': None,
 'result_dir': PosixPath('results/distilbert-base-uncased'),
 'debug': False,
 'seed': 20210425,
 'train': {'max_epochs': 100,
  'SAM': False,
  'folds': [1, 2],
  'checkpoints': ['results/distilbert-base-uncased/distilbert-base-uncased-focal_1.pth',
   None],
  'loss': {'name': 'focal', 'gamma': 2},
  'optimizer': {'name': 'AdamW'},
  'finetune': {'do': True, 'step1_epochs': 2, 'step2_epochs': 4},
  'lr': 0.0001,
  'scheduler': {'name': 'ReduceLROnPlateau',
   'params': {'factor': 0.5, 'patience': 3, 'verbose': True}}},
 'dataset': {'dir': PosixPath('data/ori'), 'batch_size': 40, 'num_workers': 8}}

In [5]:
trainer = MyTrainer(C, 1, "results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05_1.pth")

Load pretrained results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05_1.pth


In [6]:
model = trainer.model
model.eval()
torch.set_grad_enabled(False)

In [42]:
# model.load_state_dict(torch.load("results/distilbert-base-uncased/distilbert-base-uncased-focal-AdamW-lr1e-05_1.pth")["model"])

<All keys matched successfully>

In [7]:
ds = trainer.dl_test.dataset
ds2 = trainer.dl_test2.dataset
tds = trainer.tdl.dataset
vds = trainer.vdl.dataset

In [8]:
id, text, level, otext = tds[16]
print(id, level)
print(otext)

r = model(text[None].cuda())[0].softmax(1)[0].cpu()
print(r, r.argmax())

19 tensor(0)
kibana: {"type":"error","tags":["warning","stats-collection"],"level":"error","error":{"message":"No Living connections","name":"Error","stack":"Error: No Living connections\n at sendReqWithConnection (/usr/share/kibana/node_modules/elasticsearch/src/lib/transport.js:226:15)\n at next (/usr/share/kibana/node_modules/elasticsearch/src/lib/connection_pool.js:214:7)\n at process._tickCallback (internal/process/next_tick.js:61:11)"},"message":"No Living connections"}
tensor([0.1148, 0.1784, 0.1768, 0.1083, 0.1441, 0.0716, 0.2060]) tensor(6)


In [9]:
for ids, texts, tlevels, otexts in trainer.tdl:
    break

In [17]:
(model(texts.cuda())[0].cpu().argmax(1) == tlevels).sum() / len(ids) * 100

tensor(20.)

In [18]:
model(texts.cuda())[0].cpu().argmax(1) == tlevels

tensor([False,  True, False, False,  True, False, False, False, False,  True,
         True, False, False, False,  True, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False, False,
         True, False, False, False, False, False, False, False,  True, False])